### Extracción de Datos

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
%pip install scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

Note: you may need to restart the kernel to use updated packages.


In [14]:
url_data = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"
print ("Carga de datos exitosa")

Carga de datos exitosa


In [15]:
try:
    respuesta = requests.get(url_data)
    respuesta.raise_for_status()  # Verifica si la solicitud fue exitosa (código 200)
    datos = pd.DataFrame(respuesta.json())
    print("Datos cargados correctamente.")
except requests.exceptions.RequestException as e:
    print(f"Error al cargar los datos: {e}")

Datos cargados correctamente.


In [18]:
datos

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
...,...,...,...,...,...,...
7262,9987-LUTYD,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
7263,9992-RRAMN,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
7264,9992-UJOEL,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
7265,9993-LHIEB,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Two year', 'PaperlessBilling': '..."


In [19]:
#Configuración de estilo para los gráficos
sns.set(style="whitegrid")
plt.rcParams['figure.dpi'] = 100  # Aumenta la resolución de los gráficos
plt.rcParams['font.size'] = 12  # Tamaño de fuente por defecto
print("Configuración de estilo aplicada para los gráficos.")

Configuración de estilo aplicada para los gráficos.


### Limpieza y Procesamiento de Datos

In [21]:
# Elimina columnas que no aportan valor al análisis o a los modelos predictivos, como identificadores únicos.
num_duplicados = datos['customerID'].duplicated().sum()
if num_duplicados > 0:
    datos = datos.drop_duplicates(subset=['customerID'])
    print(f"Se eliminaron {num_duplicados} filas duplicadas basadas en 'customerID'.")
else:
    print("No se encontraron filas duplicadas basadas en 'customerID'.")

No se encontraron filas duplicadas basadas en 'customerID'.


In [ ]:
# Identificar y acomodar columnas basado en el diccionario de datos.



### Análisis Gráficos de Variables Cuantitativas o Numéricas

***Distribución del Abandono***

In [7]:
datos.groupby('Churn').value_counts

<bound method DataFrameGroupBy.value_counts of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000021BD8767380>>